In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [76.8 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/

In [3]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-03-07 17:44:47--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.83MB/s    in 0.2s    

2022-03-07 17:44:47 (5.83 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Jewelry_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   50423057|R135Q3VZ4DQN5N|B00JWXFDMG|     657335467|Everbling Purple ...|         Jewelry|          5|            0|          0|   N|                Y|           Beauties!|so beautiful even...| 2015-08-31|
|         US|   11262325|R2N0QQ6R4T7YRY|B00W5T1H9W|      26030170|925 Sterling Silv...|         Jewelry|          5|    

In [6]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [38]:
#remove null data
print(df.count())
df =df.dropna();
print(df.count())

1767753
1767394


### Create DataFrames to match tables

In [39]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
#disp as pandas, better reading
df.limit(5).toPandas() 

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,50423057,R135Q3VZ4DQN5N,B00JWXFDMG,657335467,Everbling Purple and Clear Briolette Drop Swar...,Jewelry,5,0,0,N,Y,Beauties!,so beautiful even tho clearly not high end ......,2015-08-31
1,US,11262325,R2N0QQ6R4T7YRY,B00W5T1H9W,26030170,925 Sterling Silver Finish 6ct Simulated Diamo...,Jewelry,5,0,0,N,N,Great product.,"Great product.. I got this set for my mother, ...",2015-08-31
2,US,27541121,R3N5JE5Y4T6W5M,B00M2L6KFY,697845240,"Sterling Silver Circle ""Friends Forever"" Infin...",Jewelry,5,0,0,N,Y,Exactly as pictured and my daughter's friend l...,Exactly as pictured and my daughter's friend l...,2015-08-31
3,US,5350721,R2I150CX5IVY9Q,B0006SW2WU,569859289,Surgical Stainless Steel Domed 9mm Fishbone Ri...,Jewelry,5,0,0,N,Y,Five Stars,Love it. Fits great. Super comfortable and nea...,2015-08-31
4,US,24484424,R1RM9ICOOA9MQ3,B009YPDW70,332947422,"Sterling Silver Family Pendant Necklace, 18""",Jewelry,5,0,0,N,Y,... a Mother's Day gift for my Mom and she lov...,Got this as a Mother's Day gift for my Mom and...,2015-08-31


In [40]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   45657423|             1|
|   47663761|             1|
|   11931196|             1|
|   44620216|             1|
|    1167081|             1|
|   17682867|             1|
|   10782454|             1|
|   51881951|             1|
|    1537684|             1|
|   17133428|             5|
|     661760|             1|
|    9867391|             9|
|   31268594|             1|
|   48581590|             1|
|   32466409|             3|
|   38490515|             1|
|   41318181|             4|
|   18800155|             6|
|   46306239|             1|
|   28409312|             1|
+-----------+--------------+
only showing top 20 rows



In [41]:
# Create the products_table DataFrame and drop duplicates. 
# from pyspark.sql import Row
products_df=df.select(["product_id","product_title"]).drop_duplicates()
#product_id has duplicates if you load the review_id table first. therefore check if duplicates exists
#also create products_df first before review_id table and also remove duplicates in review_id table for product_id
if products_df.count() > products_df.dropDuplicates(["product_id","product_title"]).count():
    raise ValueError('Data has duplicates')
# another way of checking for duplicates on product_id
# products_df=df.exceptAll(df.dropDuplicates(["product_id","product_title"]))
# products_df.show()

# from pyspark.sql import Row
# products_df = df.dropDuplicates(["product_id"]).select("product_id", "product_title")
products_df.show(truncate=False)

+----------+---------------------------------------------------------------------------------------------------------------------------------------+
|product_id|product_title                                                                                                                          |
+----------+---------------------------------------------------------------------------------------------------------------------------------------+
|B00LHJMHVS|FAIRY COUPLE Women's Round Cubic Zirconia with Smaller Cubic Zirconia Ring R474-9                                                      |
|B00TPWQ6I4|Chic Alert Medical ID Cam044S75 Simple Strand Chain Medical ID Bracelet 7.5 in. Silver Tone                                            |
|B00P837QR4|The Original Roll-on Bracelet- Set of 3- Special Olympics Ghostbusters                                                                 |
|B00L280CN4|10pcs 18" 18 Inch 925 Silver Plated 1.2MM Snake Chain Necklace New                            

In [44]:
# Create the review_id_table DataFrame. 
# make sure to remove duplicates for the product_id because it causes issues in postgre when you have duplicates and trying
# to do product_id TEXT PRIMARY KEY NOT NULL UNIQUE will raise errors
review_id_df = df.select(["review_id", "customer_id","product_id","product_parent",to_date("review_date", 'yyyy-MM-dd').alias("review_date")]).drop_duplicates(["product_id"])
review_id_df.show(truncate=False)
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
# review_id_df = df.select([, to_date("review_date", 'yyyy-MM-dd').alias("review_date")])

+--------------+-----------+----------+--------------+-----------+
|review_id     |customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1PJCXWQ5QHC06|1123157    |1438809484|10070796      |2015-01-04 |
|R2R7EUX4QKLDHD|35299126   |B00006C7GQ|83537871      |2002-11-23 |
|R2X7ED1ZDWOIK0|35018078   |B00006JMZ8|765671386     |2003-12-14 |
|R1R5QZXLPZS81O|19221258   |B00007G9BS|563298962     |2007-08-24 |
|R3MB647O345KYY|51347326   |B00009EOU4|877548069     |2007-01-11 |
|R93K1LVNY6UJR |30396969   |B00009MEOI|942160082     |2007-09-17 |
|R277VSAX9A85RS|43193258   |B0000ATBZZ|235028588     |2008-10-04 |
|R11U3WAVOMHR27|21879428   |B0000AWSIN|781233860     |2006-02-28 |
|R3906683RK03U |12241691   |B0000AWVWC|424558521     |2009-09-07 |
|R2TM4HYGGLSMF0|24158652   |B0000AZ14W|491334200     |2009-03-05 |
|R2LAI4GYGYIRJX|52801456   |B0000B35EA|679439358     |2006-12-13 |
|R1AKHUKCZQLDY6|11115555   |B0000BUX4T|350676754     |2007-03-

In [45]:
# make sure review_id table has no product_id duplicates
if review_id_df.count() > review_id_df.dropDuplicates(["product_id"]).count():
    raise ValueError('Data has duplicates')

In [46]:
from typing import cast
# Create the vine_table. DataFrame
from pyspark.sql.types import IntegerType
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
#convert to integer type bec star_rating was a text in df but int in pgAdmin table definition
vine_df = vine_df.withColumn("star_rating",vine_df["star_rating"].cast(IntegerType()))
vine_df.show(truncate=False)

+--------------+-----------+-------------+-----------+----+-----------------+
|review_id     |star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R135Q3VZ4DQN5N|5          |0            |0          |N   |Y                |
|R2N0QQ6R4T7YRY|5          |0            |0          |N   |N                |
|R3N5JE5Y4T6W5M|5          |0            |0          |N   |Y                |
|R2I150CX5IVY9Q|5          |0            |0          |N   |Y                |
|R1RM9ICOOA9MQ3|5          |0            |0          |N   |Y                |
|R2J2KMDL10UMSH|5          |0            |0          |N   |Y                |
|R3R9ZUFA4TB4FQ|5          |0            |0          |N   |Y                |
|R3UQ8VAQN7R6WL|5          |0            |0          |N   |Y                |
|R1FXZ69C01JNQM|5          |0            |0          |N   |Y                |
|RY36LB4OW0FFS |5          |0            |0          |N   |Y    

In [47]:
vine_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)



### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [29]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://mypostgredb.cxnlhanfljb4.us-west-2.rds.amazonaws.com:5432/test"
config = {"user":"root", 
          "password": "########", 
          "driver":"org.postgresql.Driver"}

In [48]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [49]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [50]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [51]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)